# Experiments for the Energy Efficiency Master thesis in DYNAMOS

This Jupyter notebook is used for the experiments for the master thesis in DYNAMOS about energy efficiency.

FABRIC API docs: https://fabric-fablib.readthedocs.io/en/latest/index.html


## Step 1:  Configure the Environment, Create Slice, Configure Kubernetes Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

After following all steps of the Configuring Environment notebook, you should be able to run this notebook without additional steps.

Next, you will need to have setup the slice in FABRIC using the [Create Slice](../create_slice.ipynb) notebook.

Finally, configure the kubernetes environment using the [Configure Kubernetes](../k8s-setup/k8s_setup.ipynb) and [Configure Kubernetes for DYNAMOS Specifically](../k8s-setup/DYNAMOS_k8s_setup.ipynb) notebooks.

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

## Step 2: Setup the Environment for this Notebook

### Import FABRIC API and other libraries

In [ ]:
import json
import traceback
import datetime

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();


### Configure the parameters and get Node Attributes
Can be used to set the corresponding nodes and other variables used for subsequent cells.

Also, the necessary variables are fetched here, such as the slice to use.

In [ ]:
# ========== Step 1: Configure Parameters ==========
slice_name = 'DYNAMOS'
node1_name = 'Node1'
# TODO: create Node2 in create_slice.ipynb?
node2_name = 'Node2'
# TODO: more and below needed?
node1_nic_name = 'NIC1'
node2_nic_name = 'NIC2'
network_name = 'NET1'

# ========== Step 2: Get and Print Node Attributes ==========
try:
    # Get slice by name: https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.get_slice
    slice = fablib.get_slice(name=slice_name)
    # Get slice nodes
    for node in slice.get_nodes():
        print(f"{node}")
    # TODO: set nodes here as well
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

### Extend the Slice before Running Experiments
This code extends the slice's end date to two weeks (the current maximum lease time) to make sure that the slice can be used for a longer period of time.

In [ ]:
# Print slice information for debugging purposes
slice.show()

# # Calculate the end date for 2 weeks from now with timezone information to fit expected format
end_date = (datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S %z")
# Renew the slice with the calculated end date
slice.renew(end_date=end_date)

# TODO: the function is working, but it does not save in the slice information afterwards, which is a bug in FABRIC. 
# So, skip for now, a workaround is to shortly before the experiment create the slice and configure it so that I have two weeks for it.

## Step 3: TODO: further steps for the experiments here.
TODO